<a href="https://colab.research.google.com/github/Teemo37/NL_LAB_1/blob/main/LR4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive')

train_images = train_generator.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/learningData/Train',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32
)
print(len(train_images))

Mounted at /content/drive
Found 55005 images belonging to 5 classes.
1719


In [ ]:
test_images = test_generator.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/learningData/Test',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False #перемешивание изображений в алф порядке
)
test_images.image_shape

Found 15000 images belonging to 5 classes.


(224, 224, 3)

In [ ]:
val_images = train_generator.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/learningData/Val',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32
)

Found 5000 images belonging to 5 classes.


In [ ]:
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='max'
)
pretrained_model.trainable = False

9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
from keras.optimizers import Adam

x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(5, activation='softmax')(x)

model = tf.keras.Model(inputs=pretrained_model.input, outputs=outputs)

optimizer = Adam(learning_rate=0.001)

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[0][0]']      

In [ ]:
history = model.fit(
    train_images,
    validation_data=val_images,
    batch_size = 32,
    epochs=10,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=2,
            verbose=1,
            restore_best_weights=True
        )
    ]
)

model.save("models/model1")

Epoch 1/10
 512/1719 [=======>......................] - ETA: 2:20:48 - loss: 0.1596 - accuracy: 0.9443

In [ ]:
import matplotlib.pyplot as plt
# Plot the loss and accuracy curves for training and validation
plt.plot(history.history['val_loss'], color='b', label="validation loss")
plt.title("Test Loss")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
# confusion matrix
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix
# Predict the values from the test dataset
Y_pred = model.predict(test_images)


In [ ]:
Y_pred_classes = np.argmax(Y_pred,axis = 1)
Y_true = test_images.labels
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
f,ax = plt.subplots(figsize=(80, 80))
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,cmap="Greens",linecolor="gray", fmt= '.1f',ax=ax)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_true,Y_pred_classes))

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

model1 = Sequential()
#
model1.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',
                 activation ='relu', input_shape = (224,224,3)))
model1.add(MaxPool2D(pool_size=(2,2)))
model1.add(Dropout(0.25))

model1.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same',
                 activation ='relu'))
model1.add(MaxPool2D(pool_size=(2,2)))
model1.add(Dropout(0.25))
#
model1.add(Conv2D(filters = 128, kernel_size = (5,5),padding = 'Same',
                 activation ='relu'))
model1.add(MaxPool2D(pool_size=(2,2)))
model1.add(Dropout(0.25))
#
model1.add(Flatten())
model1.add(Dense(256, activation = "relu"))
model1.add(Dropout(0.25))
model1.add(Dense(36, activation = "softmax"))

print(model1.summary())

In [ ]:
optimizer = Adam(learning_rate=0.001)
model1.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath='model/model2/weignts',
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
history1 = model1.fit(
    train_images,
    validation_data=val_images,
    batch_size = 32,
    epochs=10,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            verbose=1,
            restore_best_weights=True
        ), cp_callback
    ]
)

model1.save("models/model2/themodel")

In [ ]:
import matplotlib.pyplot as plt
# Plot the loss and accuracy curves for training and validation
plt.plot(history1.history['val_loss'], color='b', label="validation loss")
plt.title("Test Loss")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
# confusion matrix
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix
# Predict the values from the test dataset
Y_pred1 = model1.predict(test_images)

In [ ]:
Y_pred_classes1 = np.argmax(Y_pred1,axis = 1)
Y_true1 = test_images.labels
confusion_mtx1 = confusion_matrix(Y_true1, Y_pred_classes1)
f,ax1 = plt.subplots(figsize=(80, 80))
sns.heatmap(confusion_mtx1, annot=True, linewidths=0.01,cmap="Greens",linecolor="gray", fmt= '.1f',ax=ax1)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_true1,Y_pred_classes1))